## Erster Numpy Test mit Listen als numpy Array

In [1]:
import numpy as np
planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
position = np.array(planet_position,  dtype=np.float32)

print (position)
print()
print(position[0][0])


[[  1.49597897e+11   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00]]

1.49598e+11


## Memory View Beispiel

In [2]:
%load_ext Cython

In [6]:
%%cython -f -c-fopenmp --link-args=-fopenmp

import numpy as np

narr = np.arange(12, dtype=np.dtype("d")).reshape((4, 3))
cdef double [:, :] narr_view = narr

narr_view[2][2] = 4128741294.36346534

print (narr)
print (narr_view)
print (narr_view[2][2])

[[  0.00000000e+00   1.00000000e+00   2.00000000e+00]
 [  3.00000000e+00   4.00000000e+00   5.00000000e+00]
 [  6.00000000e+00   7.00000000e+00   4.12874129e+09]
 [  9.00000000e+00   1.00000000e+01   1.10000000e+01]]
<MemoryView of 'ndarray' object>
4128741294.3634653


## Memory View Beispiel Test Umbau

In [ ]:
%%cython -f -c-fopenmp --link-args=-fopenmp

import numpy as np

planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
position = np.array(planet_position,  dtype=np.float32)
cdef float [:, :] position_view = position
print (position)
print (position_view)
print (position_view[0][0])
print (position_view[2][2])

position_view[2][2] = 5
print (position_view[2][2])

## Erfolgreicher NoGil Test mit Memory View

In [ ]:
%%cython -f -c-fopenmp --link-args=-fopenmp

import numpy as np
cimport numpy as cnp
cimport cython
ctypedef cnp.float32_t dtype_t

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Abstand (float [:, :] position_view , float[:] result ,int first, int second) nogil:   
    result[0] = position_view[first][0] - position_view[second][0]
    result[1] = position_view[first][1] - position_view[second][1]
    result[2] = position_view[first][2] - position_view[second][2]

planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
position = np.array(planet_position,  dtype=np.float32)
cdef float [:, :] position_view = position
cdef float [:] result1 = np.array([0,0,0],  dtype=np.float32)
cdef float [:] result2 = np.array([0,0,0],  dtype=np.float32)

Abstand(position_view, result1, 0, 1)
position_view_result1 = np.asarray(result1)

Abstand(position_view, result2, 1, 0)
position_view_result2 = np.asarray(result2)

print (position_view_result1)
print (position_view_result2)

## Build Test

In [ ]:
import os
os.mkdir('/tmp/cython_galaxy_einzel_build')

In [ ]:
%%file /tmp/cython_galaxy_einzel_build/abstand.pyx
#Abstands Funktion
#Abstand_Sonne[0] = position[1][0] - position[0][0]
cimport cython

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Abstand (float [:, :] position_view , float[:] result ,int first, int second) nogil:   
    result[0] = position_view[first][0] - position_view[second][0]
    result[1] = position_view[first][1] - position_view[second][1]
    result[2] = position_view[first][2] - position_view[second][2]

In [ ]:
%%file /tmp/cython_galaxy_einzel_build/abstand_test.pyx

import numpy as np
from abstand import Abstand 
cimport numpy as cnp
cimport cython
ctypedef cnp.float32_t dtype_t

def test():
    planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
    position = np.array(planet_position,  dtype=np.float32)
    cdef float [:, :] position_view = position
    cdef float [:] result1 = np.array([0,0,0],  dtype=np.float32)
    cdef float [:] result2 = np.array([0,0,0],  dtype=np.float32)

    Abstand(position_view, result1, 0, 1)
    position_view_result1 = np.asarray(result1)

    Abstand(position_view, result2, 1, 0)
    position_view_result2 = np.asarray(result2)

    print (position_view_result1)
    print (position_view_result2)

In [ ]:
%%file /tmp/cython_galaxy_einzel_build/start.py

from abstand_test import test
test()

In [ ]:
%%file /tmp/cython_galaxy_einzel_build/setup.py


# Aufruf: python3 setup.py build_ext --inplace
# Windows: zusaetzliche Option --compiler=mingw32
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy

ext_modules=[ Extension("abstand", ["abstand.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
              Extension("abstand_test", ["abstand_test.pyx"],
        extra_compile_args=['-O3'], libraries=['m'],
            include_dirs=[numpy.get_include()])
            ]
             
setup( name = 'cython demo',
  cmdclass = {'build_ext': build_ext},
  ext_modules = ext_modules)


## Build Script

In [ ]:
%%bash
cd /tmp/cython_galaxy_einzel_build
python3 setup.py build_ext --inplace

In [ ]:
%%bash
cd /tmp/cython_galaxy_einzel_build
python3 start.py